### 수행단계
- 사용자별 평균 정밀도 계산 코드를 개발한다.

<!-- Precision@K와 Recall@K 정의

1. Precision@K (정밀도): 추천한 영화들 중 사용자가 실제로 평가한 영화의 비율
공식: Precision@K = TP / (TP + FP)
TP (True Positive): 추천한 영화 중 사용자가 평가한 영화
FP (False Positive): 추천한 영화 중 사용자가 평가하지 않은 영화

2. Recall@K (재현율): 사용자가 실제로 평가한 영화 중 추천된 영화의 비율
공식: Recall@K = TP / (TP + FN)
TP (True Positive): 추천한 영화 중 사용자가 평가한 영화
FN (False Negative): 사용자가 평가한 영화 중 추천되지 않은 영화 -->

In [19]:
# 평균 정밀도(AP@K)란?
# - 추천 시스템의 성능을 측정하기 위해 사용되는 지표.
# - 사용자가 실제로 좋아한(평점을 매긴) 영화들 중에서 추천된 영화들이 얼마나 잘 맞아떨어졌는지를 평가.
# 
# 평균 정밀도(AP@K) 공식
# AP@K = (1 / m) * Σ(Precision@i * rel(i)) for i = 1 to K
# - m: 사용자가 좋아한(평점을 매긴) 영화의 총 개수
# - K: 추천된 영화의 개수
# - Precision@i: i번째 영화까지의 정밀도 = TP / (i + 1)
# - rel(i): i번째 영화가 사용자가 좋아했는지 여부 (1 또는 0)
#
# 목적
# - AP@K를 통해 추천된 영화 리스트가 얼마나 정확하게 사용자의 선호도를 반영하는지 측정.

In [28]:
import pandas as pd

csv_file_path = 'movie_ratings_with_users.csv'

df = pd.read_csv(csv_file_path)

# 추천된 영화 리스트
recommended_movies = ["가위손", "나 홀로 집에", "대부", "사운드 오브 뮤직"]

# 사용자별 AP@K를 계산하는 함수
def calculate_apk_with_details(df, recommended_movies, user_index):
    """
    사용자별 평균 정밀도(AP@K) 계산 및 세부 정보를 반환
    
    Parameters:
    - df: DataFrame (사용자-영화 평가 데이터)
    - recommended_movies: 추천된 영화 리스트 (가나다순)
    - user_index: 사용자의 인덱스
    
    Returns:
    - details: 사용자별 Precision@i, rel(i), 그리고 AP@K 세부 정보가 포함된 리스트
    """
    # 사용자 데이터 (해당 사용자가 평가한 모든 영화 데이터)
    user_data = df.iloc[user_index, 1:].values  # 사용자 ID 제외
    # print(user_data)
    
    # 추천된 영화의 평가 데이터
    recommended_data = df.loc[user_index, recommended_movies].values
    
    # 사용자가 좋아한(평가한) 영화의 총 개수
    m = sum(user_data)  # 1의 개수
    if m == 0:
        return {"AP@K": 0, "details": []}  # 평가한 영화가 없으면 AP@K는 0
    
    # 평균 정밀도 계산
    apk = 0
    tp = 0  # True Positive 누적 개수
    details = []
    for i in range(len(recommended_data)):
        rel_i = recommended_data[i]  # rel(i): i번째 영화가 사용자가 좋아했는지 여부
        if rel_i == 1:  # 추천된 영화가 사용자가 좋아한 영화라면
            tp += 1
            precision_at_i = tp / (i + 1)  # Precision@i = TP / (i + 1)
            apk += precision_at_i
        else:
            # rel(i)가 0이라도 정밀도(Precision)는 유효하며 계산 가능함
            precision_at_i = tp / (i + 1)  
        # 세부 정보를 리스트에 추가
        details.append({
            "K": i + 1,
            "Precision@i": precision_at_i,
            "rel(i)": rel_i
        })
    
    # AP@K 계산
    ap_at_k = apk / m
    return {"AP@K": ap_at_k, "details": details}

# 사용자별 AP@K 계산 및 세부 정보 저장
all_results = []

for user_index in range(df.shape[0]):  # 사용자 수만큼 반복
    user_id = df.iloc[user_index, 0]  # 사용자 ID
    result = calculate_apk_with_details(df, recommended_movies, user_index)
    all_results.append({
        "사용자": user_id,
        "AP@K": result["AP@K"],
        "details": result["details"]
    })

# 사용자별 상세 결과 출력
print("-" * 50)
total_users = len(all_results)  # 총 사용자 수
for idx, result in enumerate(all_results, start=1):  # 사용자 번호(idx) 추가
    print(f"사용자: {result['사용자']}")
    print(f"K(추천된 영화의 개수)={len(recommended_movies)}")
    print(f"m(사용자가 좋아한(평점을 매긴) 영화의 총 개수)={sum(df.loc[df['사용자'] == result['사용자']].iloc[0, 1:])}")
    print(f"AP@K(평균정밀도): {result['AP@K']:.6f}")
    print("Details:")
    for detail in result["details"]:
        print(f"K={detail['K']}, Precision@{detail['K']}={detail['Precision@i']:.6f}, rel({detail['K']})={detail['rel(i)']}")
    print(f"{'-' * 50} ({idx}/{total_users})")  # 사용자 번호와 총 사용자 수 출력





--------------------------------------------------
사용자: 1917
K(추천된 영화의 개수)=4
m(사용자가 좋아한(평점을 매긴) 영화의 총 개수)=7
AP@K(평균정밀도): 0.345238
Details:
K=1, Precision@1=1.000000, rel(1)=1
K=2, Precision@2=0.500000, rel(2)=0
K=3, Precision@3=0.666667, rel(3)=1
K=4, Precision@4=0.750000, rel(4)=1
-------------------------------------------------- (1/10)
사용자: 10418
K(추천된 영화의 개수)=4
m(사용자가 좋아한(평점을 매긴) 영화의 총 개수)=5
AP@K(평균정밀도): 0.383333
Details:
K=1, Precision@1=0.000000, rel(1)=0
K=2, Precision@2=0.500000, rel(2)=1
K=3, Precision@3=0.666667, rel(3)=1
K=4, Precision@4=0.750000, rel(4)=1
-------------------------------------------------- (2/10)
사용자: 1980
K(추천된 영화의 개수)=4
m(사용자가 좋아한(평점을 매긴) 영화의 총 개수)=6
AP@K(평균정밀도): 0.666667
Details:
K=1, Precision@1=1.000000, rel(1)=1
K=2, Precision@2=1.000000, rel(2)=1
K=3, Precision@3=1.000000, rel(3)=1
K=4, Precision@4=1.000000, rel(4)=1
-------------------------------------------------- (3/10)
사용자: 2277
K(추천된 영화의 개수)=4
m(사용자가 좋아한(평점을 매긴) 영화의 총 개수)=6
AP@K(평균정밀도): 0.458333

In [29]:
# 전체 사용자의 AP@K 테이블 생성
apk_table = pd.DataFrame({
    "사용자": [result["사용자"] for result in all_results],
    "AP@K (평균 정밀도)": [result["AP@K"] for result in all_results]
})

# 전체 사용자의 평균 정밀도 테이블 출력
apk_table

,사용자,AP@K (평균 정밀도)
0,1917,0.345238
1,10418,0.383333
2,1980,0.666667
3,2277,0.458333
4,1805,0.345238
5,5136,0.402778
6,1561,0.277778
7,1105,0.600000
8,1312,0.400000
9,3189,0.277778
